# WRC Scraper  2020


*TO DO  - add in database elements. Schema will be in `wrcResults2020.sql`.*

There may be duplication of data across various objects in the final class. This is for convenience as much as anything, trying to preserve both captures of the original data with combined or derived data, as well as trying to put the data in places we can easily find it. Consistency in a live setting is where this may fall apart!

*Add in `asyncio` scheduler elements to call WRC API regularly. Avoid race conditions by scheduling items together in the same scheduled event if they compete for the same write resource.*

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from WRC_2020_scraper import *
from WRC_2020_scraper import getSeasonCategories

In [3]:

# TO DO - this should go into a general utils package
def _jsInt(val):
    """Ensure we have a JSON serialisable value for an int.
       The defends against non-JSON-serialisable np.int64."""
    try:
        val = int(val)
    except:
        val = None
        
    return val

In [4]:
#paths=[]
#parts=attr.split('.')
#for i, _ in enumerate(parts):
#    if i:
#        paths.append('.'.join(parts[:-i]))
#paths.reverse()

def _checkattr(obj,attr):
    """Check an object exists and is set to a non-null value."""
    
    #TO DO  - support attributes done a path, checking each step in turn
    
    if hasattr(obj,attr):
        objattr = getattr(obj, attr)
        if isinstance(objattr, pd.DataFrame):
            return not objattr.empty
        elif objattr:
            return True
        
    return False

In [5]:
# TO DO - define a class for each table
import warnings

#https://stackoverflow.com/questions/28208949/log-stack-trace-for-python-warning/29567225
import traceback

_formatwarning = warnings.formatwarning

The following generates stacktrace warnings useful when debugging in IPython notebook kernel environment.

In [6]:
def formatwarning_tb(*args, **kwargs):
    s = _formatwarning(*args, **kwargs)
    
    _tb = traceback.format_stack()
    useful=False
    tb=[]
    for i in _tb:
        if 'ipython-input' in i:
            tb.append(i)
    s += ''.join(tb[:-1])
    return s

warnings.formatwarning = formatwarning_tb
#logging.captureWarnings(True)

    
class IPythonWarner:
    """Tools for reporting warnings."""
    def __init__(self,nowarn=False):
        self.nowarn = nowarn or None
    
    def warner(self, cond, msg, nowarn=None):
        """Test a condition and if True display a warning message."""
        if nowarn is None:
            nowarn=self.nowarn
            
        if not nowarn and cond:
            warnings.warn(msg)
        

In [161]:
class WRCBase(IPythonWarner):
    "Base class for all WRC stuff."
    def __init__(self, nowarn=True):
        IPythonWarner.__init__(self, nowarn=nowarn)
        
    def _null():
        pass

In [162]:
class WRCSeasonBase(WRCBase):
    """Base class for things related to with seasons."""
    def __init__(self, season_external_id=None, autoseed=False, nowarn=True):
        WRCBase.__init__(self, nowarn=nowarn)
        
        self.season_external_id = _jsInt(season_external_id)
        if not self.season_external_id and autoseed:
            self._check_season_external_id()
            
    def _check_season_external_id(self, season_external_id=None):
        """Check that season_external_id exists and if not, get one."""
        self.season_external_id =  _jsInt(season_external_id) or self.season_external_id
        if not _checkattr(self,'season_external_id'):
            #Get current one from active rally
            #It's also available from current_season_events
            event, days, channels = getActiveRally()
            self.event, self.days, self.channels = event, days, channels
            #The returned np.int64 is not JSON serialisable
            self.season_external_id = _jsInt(event.loc[0,'season.externalId'])



In [163]:
WRCSeasonBase(autoseed=True)#.season_external_id

In [164]:
class WRCSeasonCategories(WRCSeasonBase):
    def __init__(self, autoseed=False, nowarn=True):
        WRCSeasonBase.__init__(self, nowarn=nowarn)
        
        self.season_categories = None
        self.championship_codes = None
        
        if autoseed:
            self.fetchSeasonCategories()
            
    def fetchSeasonCategories(self):
        self.season_categories = getSeasonCategories()
        self.championship_codes = getSeasonChampionshipCodes()

In [165]:
zz = WRCSeasonCategories()
zz.fetchSeasonCategories()
zz.championship_codes

,id,category.name,drivers,codrivers,manufacturers
0,35,WRC,37,38,39
1,46,WRC 2,40,41,43
2,49,WRC 3,44,45,None
3,58,JWRC,46,47,None


In [166]:
class WRCChampionship(WRCSeasonCategories):       
    """Class for championship."""
    def __init__(self, category='WRC', typ='drivers',
                 season_external_id = None,
                 autoseed=False, nowarn=True ):
        WRCSeasonCategories.__init__(self, autoseed=False, nowarn=True)

        self.championships = {}
        
        if autoseed:
            fetchChampionship(self, category=category, typ=typ,
                              season_external_id=season_external_id)
        
    def fetchChampionship(self, category='WRC',typ='drivers', season_external_id=None):
        self._check_season_external_id(season_external_id)
        _c = self.championships
        if category not in _c:
            _c[category] = {}
        _cc = _c[category]
        if typ not in _cc:
            _cc[typ] =  {}
        _cct = _cc[typ]
        (_cct['championship'], _cct['championshipRounds'], \
         _cct['championshipEntries']) = getChampionship(category=category,
                                                        typ=typ, season_external_id=season_external_id)

In [167]:
zz = WRCChampionship()
zz.fetchChampionship()

In [168]:
class WRCChampionshipStandings(WRCSeasonCategories):       
    """Standings for a championship."""
    def __init__(self, category='WRC', typ='drivers',
                 season_external_id = None,
                 autoseed=False, nowarn=True ):
        WRCSeasonCategories.__init__(self, autoseed=False, nowarn=True)
        
        self.championship_standings = {}
        
        if autoseed:
            fetchChampionshipStandings(self,category=category, typ=typ,
                                       season_external_id=season_external_id)
            
        
    def fetchChampionshipStandings(self, category='WRC',typ='drivers', season_external_id=None):
        self._check_season_external_id(season_external_id)
        _c = self.championship_standings
        if category not in _c:
            _c[category] = {}
        _cc = _c[category]
        if typ not in _cc:
            _cc[typ] =  {}
        _cct = _cc[typ]
        (_cct['championship_standings'], \
         _cct['round_results']) = getChampionshipStandings(category=category,
                                                           typ=typ, season_external_id=season_external_id)

In [169]:
zz = WRCChampionshipStandings()
zz.fetchChampionshipStandings('JWRC')
zz.fetchChampionshipStandings('WRC')
zz.championship_standings

{'JWRC': {'drivers': {'championship_standings': Empty DataFrame
   Columns: [championshipId]
   Index: [], 'round_results': Empty DataFrame
   Columns: []
   Index: []}},
 'WRC': {'drivers': {'championship_standings':     championshipEntryId  overallPosition  overallPoints championshipId
   0                   751                1             30             37
   1                   757                2             22             37
   2                   753                3             17             37
   3                   758                4             13             37
   4                   760                5             10             37
   5                   761                6              8             37
   6                   765                7              7             37
   7                   762                8              6             37
   8                   764                9              2             37
   9                   766               10  

In [16]:
# TO DO - need a more general season events class?
# If, that is, we can we look up arbitrary season events...
class WRCActiveSeasonEvents:
    """Class for Season events."""
    def __init__(self, autoseed=False):
        
        if autoseed:
            self.fetchActiveSeasonEvents()
            
    def fetchActiveSeasonEvents(self):
        self.current_season_events, self.eventdays, self.eventchannel = getActiveSeasonEvents()


In [17]:
WRCActiveSeasonEvents(autoseed=True).current_season_events.head()

,id,name,externalIdRally,externalIdEvent,timezone,active,countdown,jwrc,images.format16x9.320x180,images.format16x9.160x90,...,winner.birthDate,winner.birthPlace,winner.debutDate,winner.debutPlace,winner.website,winner.driverImageFormats,winner.externalId,winner.page,winner,seasonYear
0,100,Rallye Monte Carlo,153,124,1,True,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,1988-06-16,Belgium,2009-01-12,Rally de Portugal,https://www.thierryneuville.com/,"[{'id': 6, 'title': 'Format 16:9', 'imageForma...",762.0,NaN,NaN,2020
1,102,Rally Sweden,154,125,2,False,False,True,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
2,107,Rally Guanajuato Mexico,155,126,-6,False,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
3,114,Rally Argentina,156,127,-3,False,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
4,116,Rally de Portugal,157,128,1,False,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020


In [18]:
class WRCRally_sdb(WRCBase):
    """Base class for things with an sdbRallyId.
       Can also help find an active sdbRallyId"""
    def __init__(self, sdbRallyId=None,
                 autoseed=False, nowarn=True,):
        WRCBase.__init__(self, nowarn=nowarn)
        
        self.warner(not sdbRallyId, "sdbRallyId should really be set...")
        
        self.sdbRallyId = _jsInt(sdbRallyId)
        
        if autoseed:
            self._checkRallyId(sdbRallyId)
    
    def _checkRallyId(self, sdbRallyId=None):
        """Return a rally ID or lookup active one."""
        
        sdbRallyId = _jsInt(sdbRallyId) or self.sdbRallyId
        if not hasattr(self, 'sdbRallyId') or not self.sdbRallyId:
            self.activerally = WRCActiveRally()
            self.sdbRallyId = self.activerally.sdbRallyId
            self.name = self.activerally.name
        return self.sdbRallyId

In [19]:
WRCRally_sdb(nowarn=False).nowarn

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: sdbRallyId should really be set...
  File "<ipython-input-19-8e2761af82a9>", line 1, in <module>
    WRCRally_sdb(nowarn=False).nowarn
  File "<ipython-input-18-5d48992a5f96>", line 8, in __init__
    self.warner(not sdbRallyId, "sdbRallyId should really be set...")
  File "<ipython-input-6-e04c1be6ee32>", line 28, in warner
    warnings.warn(msg)


In [20]:
class WRCLive(WRCBase):
    """Base class for live rallies."""
    def __init__(self, live=False):
        WRCBase.__init__(self)
        
        self.live = live

In [21]:
class WRCActiveRally(WRCRally_sdb, WRCLive):
    """Class for the active rally."""
    def __init__(self, live=False ):
        WRCRally_sdb.__init__(self, nowarn=True)
        WRCLive.__init__(self, live=live)

        self.fetchData()
        
    def fetchData(self):
        event, days, channels = getActiveRally()
        self.event, self.days, self.channels = event, days, channels

        #np.int64 is not JSON serialisable
        self.sdbRallyId = int(event.loc[0,'id'])

        self.name = event.loc[0,'name']

In [191]:
WRCActiveRally().sdbRallyId

100

In [23]:
zz = WRCRally_sdb(autoseed=True)
print(zz.sdbRallyId)

100


We use the `.fetchData()` method so as to ry not to be greedy. This way, we can define a class and start to work towards only grabbling the data if we need it.

In [24]:
class WRCRetirements(WRCRally_sdb):
    """Callable class for retirements"""
    def __init__(self, sdbRallyId=None, live=False, autoseed=False):
        """Initialise retirements class."""
        WRCRally_sdb.__init__(self, sdbRallyId=sdbRallyId, autoseed=autoseed)
            
        self.retirements=None
        
        if self.sdbRallyId:
            self.fetchRetirements(self.sdbRallyId)
        
    def fetchRetirements(self, sdbRallyId=None):
        """Fetch the data from WRC API."""
        self._checkRallyId(sdbRallyId)
        self.retirements = getRetirements(self.sdbRallyId)
    
    def __call__(self):
        return self.retirements

In [25]:
zz=WRCRetirements(autoseed=True)
zz.retirements.head(3)

,retirementId,controlId,entryId,reason,retirementDateTime,retirementDateTimeLocal,status
0,1475,6591,20710,OFF ROAD,2020-01-23T20:40:00Z,0001-01-01T00:00:00+00:00,Temporary
1,1476,6588,20687,MECHANICAL,2020-01-23T21:57:00Z,0001-01-01T00:00:00+00:00,Temporary
2,1477,6591,20750,OFF ROAD,2020-01-23T22:51:00Z,0001-01-01T00:00:00+00:00,Permanent


In [26]:
class WRCPenalties(WRCRally_sdb):
    """Callable class for penalties."""
    def __init__(self, sdbRallyId=None, live=False, autoseed=False):
        """Initialise penalties class."""
        WRCRally_sdb.__init__(self, sdbRallyId=sdbRallyId, autoseed=autoseed)
            
        self.penalties=None
        
        if self.sdbRallyId:
            self.fetchPenalties(self.sdbRallyId)

    
    def fetchPenalties(self, sdbRallyId=None):
        """Fetch the data from WRC API."""
        self._checkRallyId(sdbRallyId)
        self.penalties = getPenalties(self.sdbRallyId)
    
    def __call__(self):
        return self.penalties

In [27]:
zz=WRCPenalties(autoseed=True)
zz.penalties.head(3)

,penaltyId,controlId,entryId,penaltyDurationMs,penaltyDuration,reason
0,725,6592,20730,10000,PT10S,FALSE START
1,726,6592,20753,10000,PT10S,FALSE START
2,727,6590,20760,10000,PT10S,1 MIN LATE


In [31]:
class WRCStagewinners(WRCRally_sdb):
    """Callable class for penalties."""
    def __init__(self, sdbRallyId=None, live=False, autoseed=False):
        """Initialise penalties class."""
        WRCRally_sdb.__init__(self, sdbRallyId=sdbRallyId, autoseed=autoseed)
            
        self.stagewinners=None
        
        if self.sdbRallyId:
            self.fetchStagewinners(self.sdbRallyId)

    
    def fetchStagewinners(self, sdbRallyId=None):
        """Fetch the data from WRC API."""
        self._checkRallyId(sdbRallyId)
        self.stagewinners = getStagewinners(self.sdbRallyId)
    
    def __call__(self):
        return self.stagewinners

In [34]:
zz=WRCStagewinners()
zz.fetchStagewinners()
zz.stagewinners.head()

,stageId,entryId,stageName,elapsedDurationMs,elapsedDuration
0,1538,20684,Bayons - Bréziers,983700,00:16:23.7000000
1,1528,20685,Malijai - Puimichel (Live TV),593400,00:09:53.4000000
2,1533,20686,Curbans - Venterol 1,802000,00:13:22
3,1534,20686,Saint-Clément - Freissinières 1,703300,00:11:43.3000000
4,1535,20686,Avançon - Notre-Dame-du-Laus 1,780700,00:13:00.7000000


,penaltyId,controlId,entryId,penaltyDurationMs,penaltyDuration,reason
0,725,6592,20730,10000,PT10S,FALSE START
1,726,6592,20753,10000,PT10S,FALSE START


In [36]:
class WRCItinerary(WRCRally_sdb, WRCLive):
    """Class for WRC2020 Itinerary."""
    def __init__(self, sdbRallyId=None, live=False, autoseed=False):
        """Initialise itinerary class."""
        WRCRally_sdb.__init__(self, sdbRallyId=sdbRallyId, autoseed=autoseed)
        WRCLive.__init__(self, live=live)
        
        self.itinerary=None
        self.legs=None
        self.sections=None
        self.controls=None
        self.stages=None
        
        if self.sdbRallyId:
            self.fetchItinerary(sdbRallyId)
    
    def _checkItinerary(self):
        """Check itinerary.
           If rally not known, use active rally.
           Also set a default startListId."""
        
        _itinerary_items = ['itinerary', 'legs', 'sections', 'controls', 'stages']
        if not any([hasattr(self, i) for i in _itinerary_items]):
            self.fetchItinerary()

    
    def fetchItinerary(self, sdbRallyId=None):
        """Fetch the data from WRC API."""
        self._checkRallyId(sdbRallyId)
        self._checkItinerary()
        self.itinerary, self.legs, self.sections, self.controls, self.stages = getItinerary(self.sdbRallyId)
        

In [37]:
print(WRCItinerary(autoseed=True).sdbRallyId)

100


In [38]:
WRCItinerary(sdbRallyId=100).legs

,itineraryLegId,itineraryId,startListId,name,legDate,order,status
0,273,240,451,Thursday 23rd January,2020-01-23,1,Completed
1,272,240,452,Friday 24th January,2020-01-24,2,Completed
2,275,240,454,Saturday 25th January,2020-01-25,3,Completed
3,274,240,456,Sunday 26th January,2020-01-25,4,Completed


In [39]:
class WRCStartlist(WRCLive):
    """Class for WRC2020 Startlist table."""
    def __init__(self, startListId=None, live=False, autoseed=True, nowarn=False):
        WRCLive.__init__(self, live=live)
        
        self.startListId = _jsInt(startListId)
        
        self.startList = None
        self.startListItems = None
        
        if not nowarn and not self.startListId:
            warnings.warn("startListId should really be set..")
        
        if self.startListId or autoseed:
            self.fetchStartList(self.startListId)
    
    def _checkStartListId(self, startListId=None):
        """Return a startlistId or look one up."""
        self.startListId = startListId or self.startListId
        if not self.startListId:
            if not _checkattr(self, 'itinerary'):
                self.itinerary = WRCItinerary(autoseed=True)
                self.sdbRallyId = self.itinerary.sdbRallyId
            self.startListId = int(self.itinerary.legs.loc[0,'startListId'])
        return self.startListId
        
        
    def fetchStartList(self, startListId=None):
        self._checkStartListId(startListId)
        startList,startListItems = getStartlist(self.startListId)
        self.startList, self.startListItems = startList,startListItems

In [40]:
zz = WRCStartlist(autoseed=True)
zz.fetchStartList()
getStartlist(451)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: startListId should really be set..
  if sys.path[0] == '':
  File "<ipython-input-40-2786fc75f2ce>", line 1, in <module>
    zz = WRCStartlist(autoseed=True)
  File "<ipython-input-39-d26a47893005>", line 12, in __init__
    warnings.warn("startListId should really be set..")


(   startListId  eventId publishedStatus      name
 0          451      124       Published  Thursday,
     startListItemId  startListId  entryId         startDateTime  \
 0             20891          451    20745  2020-01-23T17:37:00Z   
 1             20892          451    20743  2020-01-23T17:36:00Z   
 2             20893          451    20722  2020-01-23T17:35:00Z   
 3             20894          451    20741  2020-01-23T17:34:00Z   
 4             20895          451    20740  2020-01-23T17:33:00Z   
 ..              ...          ...      ...                   ...   
 83            20974          451    20702  2020-01-23T16:55:00Z   
 84            20975          451    20703  2020-01-23T16:54:00Z   
 85            20976          451    20707  2020-01-23T16:53:00Z   
 86            20977          451    20728  2020-01-23T17:15:00Z   
 87            20978          451    20683  2020-01-23T16:00:00Z   
 
            startDateTimeLocal  order  
 0   2020-01-23T18:37:00+01:00     64  

In [41]:
class WRCCars(WRCRally_sdb):
    """Class for WRC2020 Cars table."""
    def __init__(self, sdbRallyId=None, live=False, autoseed=False):  
        WRCRally_sdb.__init__(self, sdbRallyId=sdbRallyId, autoseed=autoseed)
        
        self.cars=None
        self.classes=None
        
        if self.sdbRallyId:
            self.fetchCars(sdbRallyId)
            
    def fetchCars(self, sdbRallyId=None):
        self._checkRallyId(sdbRallyId)
        cars, classes = getCars(self.sdbRallyId)
        self.cars, self.classes = cars, classes

In [42]:
WRCCars(autoseed=True)

In [206]:
class WRCRally(WRCRally_sdb):
    """Class for WRC2020 Rally table. This gives external ids."""
    def __init__(self, sdbRallyId=None, autoseed=False):  
        WRCRally_sdb.__init__(self, sdbRallyId=sdbRallyId, autoseed=autoseed)
        
        self.rally=None
        self.eligibilities=None
        self.groups=None
        
        if self.sdbRallyId:
            self.fetchRally(sdbRallyId)
            
    def fetchRally(self, sdbRallyId=None):
        self._checkRallyId(sdbRallyId)
        (self.rally, self.eligibilities, self.groups) = getRally(self.sdbRallyId)

In [207]:
zz=WRCRally(autoseed=True)
display(zz.rally)
zz.eligibilities

,externalIdRally,externalIdEvent,itineraryId,name,isMain,eventClasses,sdbRallyId
0,153,124,240,WRC,True,None,100


,0,externalIdRally,sdbRallyId
0,M,153,100
1,None,153,100
2,WRC2,153,100
3,WRC3,153,100
4,RGT,153,100


In [227]:

# TO DO - have a check stages function to get some data...

class WRCRally_stages(WRCRally_sdb, WRCLive):
    """Class referring to all rally stages."""
    def __init__(self, sdbRallyId=None, live=False,
                 autoseed=False, nowarn=True,):
        WRCRally_sdb.__init__(self, sdbRallyId=sdbRallyId, autoseed=autoseed, nowarn=nowarn)
        WRCLive.__init__(self, live=live)
        
        self.sdbRallyId = _jsInt(sdbRallyId)
        self.stages = None
        
        if autoseed:
            self._checkStages(self.sdbRallyId)

    def _checkStages(self, sdbRallyId=None):
        """Return a stages list or lookup list for active rally."""
        #Have we got an sdbRallyId?
        if not hasattr(self, 'sdbRallyId') or not self.sdbRallyId:
            self.activerally = WRCActiveRally()
            self.sdbRallyId = self.activerally.sdbRallyId
            self.name = self.activerally.name

        #The stages are in the stages return value from the itinerary
        #itinerary, legs, sections, controls, stages = getItinerary(sdbRallyId)
        if not hasattr(self, 'itinerary') or not _checkattr(self,'itinerary.stages'):
            self.itinerary = WRCItinerary(self.sdbRallyId, autoseed=True)

        _ccols=['code']+(list(set(self.itinerary.controls.columns) - set(self.itinerary.stages.columns)))
        self.stages=self.itinerary.stages.merge(self.itinerary.controls[_ccols], on='code')
        #return (self.sdbRallyId, self.stages)
    
    def lastCompletedStage(self):
        # need to check etc
        return self.stages[self.stages['status']=='Completed'].iloc[-1]['stageId']
    

In [209]:
zz=WRCRally_stages(autoseed=True)
zz.itinerary.stages # stages / controls
zz._checkStages()[1].head()
zz.lastCompletedStage()

1526

In [228]:
# Does this actually do anything other than possible checks?

class WRCRally_stage(WRCRally_stages):
    """Base class for things with a stageId.
       Can also help find a stageId list for a given rally."""
    def __init__(self, sdbRallyId=None, stageId=None, live=False,
                 autoseed=False, nowarn=True):
        WRCRally_stages.__init__(self, sdbRallyId=sdbRallyId,
                                 live=live, autoseed=autoseed, nowarn=nowarn)
        
        if not nowarn:
            if not sdbRallyId:
                warnings.warn("sdbRallyId should really be set...")
            if not stageId:
                warnings.warn("stageId should really be set...")

        stageId = _jsInt(stageId)
        
        if autoseed:
            fetchData(self.sdbRallyId, stageId)
 
    def _checkStageId(self, sdbRallyId=None, stageId=None ):
        """Return a stage ID or lookup a current one."""
        
        self._checkRallyId(sdbRallyId)
    
        stageId = _jsInt(stageId)
        

        #sdbRallyId = sdbRallyId or self.sdbRallyId
        #if not hasattr(self, 'sdbRallyId') or not self.sdbRallyId:
        #    self.activerally = WRCActiveRally()
        #    self.sdbRallyId = self.activerally.sdbRallyId
        #    self.name = self.activerally.name
        #return self.sdbRallyId
        pass

In [211]:
zz=WRCRally_stage()

In [229]:
class WRCOverall(WRCRally_stage):
    """Class for overall stage table."""
    def __init__(self, sdbRallyId=None, stageId=None, live=False,
                 autoseed=False, nowarn=True):
        WRCRally_stage.__init__(self, sdbRallyId=sdbRallyId, stageId=stageId,
                                live=live, autoseed=autoseed, nowarn=nowarn)
        
        self.overall={}

        if stageId:
            self.fetchOverall(self.sdbRallyId, stageId)

        if autoseed:
            pass

    def fetchOverall(self, sdbRallyId=None, stageId=None):
        """Fetch the data from WRC API."""
        self._checkRallyId(sdbRallyId)
        self._checkStageId(self.sdbRallyId, stageId)

        if stageId:
            self.overall[stageId] = getOverall(self.sdbRallyId, stageId)
    
    def __call__(self):
        return self.overall

In [213]:
zz=WRCOverall(stageId = 1528)
zz.fetchOverall()
zz.overall

{1528:     entryId  stageTimeMs   stageTime  penaltyTimeMs penaltyTime  totalTimeMs  \
 0     20685       593400   PT9M53.4S              0        PT0S       593400   
 1     20683       595200   PT9M55.2S              0        PT0S       595200   
 2     20686       595300   PT9M55.3S              0        PT0S       595300   
 3     20684       599800   PT9M59.8S              0        PT0S       599800   
 4     20690       603600   PT10M3.6S              0        PT0S       603600   
 ..      ...          ...         ...            ...         ...          ...   
 81    20768       880800  PT14M40.8S              0        PT0S       880800   
 82    20769       880800  PT14M40.8S              0        PT0S       880800   
 83    20770       880800  PT14M40.8S              0        PT0S       880800   
 84    20753       880800  PT14M40.8S          10000       PT10S       890800   
 85    20710      1229100  PT20M29.1S              0        PT0S      1229100   
 
      totalTime  pos

In [230]:
class WRCStageTimes(WRCRally_stage):
    """Class for stage times table."""
    def __init__(self, sdbRallyId=None, stageId=None, live=False,
                 autoseed=False, nowarn=True):
        WRCRally_stage.__init__(self, sdbRallyId=sdbRallyId, stageId=stageId,
                                live=live, autoseed=autoseed, nowarn=nowarn)
                         
        self.stagetimes={}

        if stageId:
            self.fetchStageTimes(self.sdbRallyId, stageId)

        if autoseed:
            pass

    def fetchStageTimes(self, sdbRallyId=None, stageId=None):
        """Fetch the data from WRC API."""
        self._checkRallyId(sdbRallyId)
        self._checkStageId(self.sdbRallyId, stageId)
        
        if stageId:
            self.stagetimes[stageId] = getStageTimes(self.sdbRallyId, stageId)
    
    def __call__(self):
        return self.stagetimes

In [215]:
zz=WRCStageTimes(stageId = 1528)
zz.fetchStageTimes()
zz.stagetimes

{1528:     stageTimeId  stageId  entryId  elapsedDurationMs   elapsedDuration  \
 0         85682     1528    20685           593400.0  00:09:53.4000000   
 1         85717     1528    20683           595200.0  00:09:55.2000000   
 2         85684     1528    20686           595300.0  00:09:55.3000000   
 3         85680     1528    20684           599800.0  00:09:59.8000000   
 4         85712     1528    20690           603600.0  00:10:03.6000000   
 ..          ...      ...      ...                ...               ...   
 83        85606     1528    20769           880800.0  00:14:40.8000000   
 84        85720     1528    20770           880800.0  00:14:40.8000000   
 85        85676     1528    20710          1229100.0  00:20:29.1000000   
 86        85724     1528    20750          1329900.0  00:22:09.9000000   
 87        85643     1528    20741                NaN              None   
 
        status    source  position  diffFirstMs         diffFirst  diffPrevMs  \
 0   Comple

In [231]:
class WRCSplitTimes(WRCRally_stage):
    """Class for SplitTimes stage table."""
    def __init__(self, sdbRallyId=None, stageId=None, live=False, autoseed=False):  
        WRCRally_stage.__init__(self, sdbRallyId=sdbRallyId, stageId=stageId,
                                 live=live, autoseed=autoseed)
        
        self.splitPoints = {}
        self.entrySplitPointTimes ={}
        self.splitPointTimes = {}
        
        if stageId:
            self.fetchSplitTimes(self.sdbRallyId, stageId)

        if autoseed:
            pass
            
    def fetchSplitTimes(self, sdbRallyId=None, stageId=None):
        self._checkRallyId(sdbRallyId)
        self._checkStageId(self.sdbRallyId, stageId)
        if stageId:
            (self.splitPoints[stageId], self.entrySplitPointTimes[stageId], \
             self.splitPointTimes[stageId]) = getSplitTimes(self.sdbRallyId, stageId)

In [217]:
zz=WRCSplitTimes(stageId = 1528)
zz.fetchSplitTimes()
zz.entrySplitPointTimes


{1528:     entryId        startDateTime         startDateTimeLocal  \
 0     20683  2020-01-23T19:38:00  2020-01-23T20:38:00+01:00   
 1     20684  2020-01-23T19:41:00  2020-01-23T20:41:00+01:00   
 2     20685  2020-01-23T19:44:00  2020-01-23T20:44:00+01:00   
 3     20686  2020-01-23T19:47:00  2020-01-23T20:47:00+01:00   
 4     20687  2020-01-23T19:50:00  2020-01-23T20:50:00+01:00   
 ..      ...                  ...                        ...   
 82    20766  2020-01-23T21:44:00  2020-01-23T22:44:00+01:00   
 83    20767  2020-01-23T21:45:00  2020-01-23T22:45:00+01:00   
 84    20768  2020-01-23T21:46:00  2020-01-23T22:46:00+01:00   
 85    20769  2020-01-23T21:47:00  2020-01-23T22:47:00+01:00   
 86    20770  2020-01-23T21:48:00  2020-01-23T22:48:00+01:00   
 
     stageTimeDurationMs stageTimeDuration  
 0                595200  00:09:55.2000000  
 1                599800  00:09:59.8000000  
 2                593400  00:09:53.4000000  
 3                595300  00:09:55.3000000  

TO DO - think about sqlite export.

In [277]:
def WRCdatagetter(func):
    def call(self):
        (attrs, func2) = func(self)
        if not attrs:
            return None
        if isinstance(attrs, str):
            attrs=[attrs]
        func2()
        if len(attrs)>1:
            return tuple(getattr(self,a) for a in attrs)
        return getattr(self,attrs[0])
    return call,

        
#This class will contain everything about a single rally
class WRCEvent(WRCItinerary, WRCCars, WRCPenalties, WRCRetirements, WRCStartlist,
               WRCRally, WRCStagewinners, WRCOverall, WRCStageTimes, WRCSplitTimes ):
    """Class for a rally event.
       Can be used to contain all the timing results data from a WRC rally weekend."""
    def __init__(self, sdbRallyId=None, stageId=None, live=False, autoseed=False, slurp=False):
        WRCItinerary.__init__(self, sdbRallyId=sdbRallyId, live=live,
                             autoseed=autoseed)
        WRCCars.__init__(self, sdbRallyId=sdbRallyId, live=live,
                             autoseed=autoseed)
        WRCPenalties.__init__(self, sdbRallyId=sdbRallyId, live=live,
                             autoseed=autoseed)
        WRCRetirements.__init__(self, sdbRallyId=sdbRallyId, live=live,
                             autoseed=autoseed)
        WRCStartlist.__init__(self, startListId=None, live=live,
                             autoseed=autoseed, nowarn=True)
        WRCRally.__init__(self, sdbRallyId=sdbRallyId, autoseed=autoseed)
        WRCStagewinners.__init__(self, sdbRallyId=sdbRallyId, autoseed=autoseed)
        WRCOverall.__init__(self, sdbRallyId=sdbRallyId, stageId=stageId, autoseed=autoseed)
        WRCStageTimes.__init__(self, sdbRallyId=sdbRallyId, stageId=stageId, autoseed=autoseed)
        WRCSplitTimes.__init__(self, sdbRallyId=sdbRallyId, stageId=stageId, autoseed=autoseed)
        
        if slurp:
            self.rallyslurper()

    
    @WRCdatagetter 
    def getRally(self):
        """Get external rally details."""
        return (['rally','eligibilities', 'groups'], self.fetchRally())
    

    @WRCdatagetter 
    def getItinerary(self):
        """Get itinerary.
           If rally not known, use active rally.
           Also set a default startListId."""
        return (['itinerary', 'legs', 'sections','controls', 'stages'],
                self.fetchItinerary())
       
    @WRCdatagetter   
    def getStartlist(self, startListId=None):
        """Get startlist.
           If no startListId provided, try to find a default."""
        return (['startList', 'startListItems'], self.fetchStartList())


    @WRCdatagetter
    def getPenalties(self):
        """Get penalties."""
        return ('penalties', self.fetchPenalties())
    
    @WRCdatagetter
    def getRetirements(self):
        """Get retirements."""
        return ('retirements', self.fetchRetirements())

    
    @WRCdatagetter
    def getStagewinners(self):
        return ('stagewinners', self.fetchStagewinners())
    
    @WRCdatagetter   
    def getCars(self):
        """Get cars."""
        return (['cars', 'classes' ], self.fetchCars())

    #TO DO - different decorator
    def getOverall(self, stageId=None):
        """Get Overall."""
        attrs = ['overall']
        self._checkRallyId()
        self._checkStages()
        
        if not stageId:
            stageId  = self.lastCompletedStage()
            
        if not all([_checkattr(self,a) for a in attrs]):
            self.fetchOverall(self.sdbRallyId, stageId)
        
        return self.overall
    
    
    def getStageTimes(self, stageId=None):
        """Get StageTimes."""
        attrs = ['stagetimes']
        self._checkRallyId()
        self._checkStages()
        
        if not stageId:
            stageId  = self.lastCompletedStage()
            
        if not all([_checkattr(self,a) for a in [attrs]]):
            self.fetchStageTimes(self.sdbRallyId, stageId)
        return self.stagetimes

    def getSplitTimes(self, stageId=None):
        """Get SplitTimes."""
        attrs = ['splitPoints','entrySplitPointTimes','splitPointTimes']
        self._checkRallyId()
        self._checkStages()
        
        if not stageId:
            stageId  = self.lastCompletedStage()
        
        if not all([_checkattr(self,a) for a in [attrs]]):
            self.fetchSplitTimes(self.sdbRallyId, stageId)

        return (self.splitPoints, self.entrySplitPointTimes, self.splitPointTimes)

    
    def rallyslurper(self):
        """Grab everything..."""
        self.getItinerary()
        self.getCars()
        self.getStartlist()
        self.getPenalties()
        self.getRetirements()
    

In [278]:
zz=WRCEvent()
zz.getOverall()

reget False


{1526:     entryId  stageTimeMs     stageTime  penaltyTimeMs penaltyTime  \
 0     20684     11457600  PT3H10M57.6S              0        PT0S   
 1     20685     11470200  PT3H11M10.2S              0        PT0S   
 2     20686     11471900  PT3H11M11.9S              0        PT0S   
 3     20688     11646600   PT3H14M6.6S              0        PT0S   
 4     20690     11714800  PT3H15M14.8S              0        PT0S   
 ..      ...          ...           ...            ...         ...   
 68    20754     17370000    PT4H49M30S         120000        PT2M   
 69    20762     17482400  PT4H51M22.4S          30000       PT30S   
 70    20698     17680700  PT4H54M40.7S              0        PT0S   
 71    20730     18208800   PT5H3M28.8S          10000       PT10S   
 72    20767     19519700  PT5H25M19.7S         170000     PT2M50S   
 
     totalTimeMs     totalTime  position  diffFirstMs     diffFirst  \
 0      11457600  PT3H10M57.6S         1            0          PT0S   
 1      11

In [286]:
zz.overall

{1526:     entryId  stageTimeMs     stageTime  penaltyTimeMs penaltyTime  \
 0     20684     11457600  PT3H10M57.6S              0        PT0S   
 1     20685     11470200  PT3H11M10.2S              0        PT0S   
 2     20686     11471900  PT3H11M11.9S              0        PT0S   
 3     20688     11646600   PT3H14M6.6S              0        PT0S   
 4     20690     11714800  PT3H15M14.8S              0        PT0S   
 ..      ...          ...           ...            ...         ...   
 68    20754     17370000    PT4H49M30S         120000        PT2M   
 69    20762     17482400  PT4H51M22.4S          30000       PT30S   
 70    20698     17680700  PT4H54M40.7S              0        PT0S   
 71    20730     18208800   PT5H3M28.8S          10000       PT10S   
 72    20767     19519700  PT5H25M19.7S         170000     PT2M50S   
 
     totalTimeMs     totalTime  position  diffFirstMs     diffFirst  \
 0      11457600  PT3H10M57.6S         1            0          PT0S   
 1      11

In [220]:
#This class needs renaming...
#What does it actually represent? An event? A live event? A set of events?

# TO DO - this presumably is wrong if we call in in 2021?
class WRC2020(WRCActiveSeasonEvents, WRCEvent):
    """Class for WRC data scrape using 2020 API."""

    def __init__(self, sdbRallyId=None, live=False, autoseed=False):
        WRCActiveSeasonEvents.__init__(self, autoseed=autoseed)
        WRCEvent.__init__(self, sdbRallyId, live)
        
        self.live = live
        
        
    def getActiveSeasonEvents(self):
        """Get active (current) season events."""
        _current_season_events_attrs = ['current_season_events',
                                         'eventdays', 'eventchannel' ]
        if not any([hasattr(self,a) for a in _current_season_events_attrs]) or not _checkattr(self,'current_season_events'):
            self.fetchActiveSeasonEvents()
        return (self.current_season_events, self.eventdays, self.eventchannel)


    
    
    
    

In [221]:
zz = WRC2020().getActiveSeasonEvents()
#zz

In [222]:
#zz = WRCEvent(slurp=True)

In [223]:
zz = WRCEvent(autoseed=True)
zz.getPenalties()

AttributeError: 'DataFrame' object has no attribute 'legs'

In [ ]:
wrc=WRC2020()
wrc.getActiveSeasonEvents()
wrc.activeseasonevents.current_season_events

In [ ]:
wrc.getStartlist()

In [ ]:
wrc.itinerary.sections

In [ ]:
itinerary, legs, sections, controls, stages = wrc.getItinerary()
startList,startListItems = getStartlist(startListId)
cars, classes = getCars(sdbRallyId)
#rally, eligibilities, groups = getRally(sdbRallyId)
#overall = getOverall(sdbRallyId, stageId)
#splitPoints, entrySplitPointTimes, splitPointTimes = getSplitTimes(sdbRallyId,stageId)
#stagetimes = getStageTimes(sdbRallyId,stageId)
#stagewinners = getStagewinners(sdbRallyId)
#penalties = getPenalties(sdbRallyId)
#retirements = getRetirements(sdbRallyId)
#championship = getChampionship()
#championship = getChampionshipStandingsLive()

In [ ]:
current_season_events, eventdays, eventchannel = wrc.getActiveSeasonEvents()

In [ ]:
event, days, channels = wrc.getActiveRally()


In [ ]:
event

In [ ]:
wrc.sdbRallyId